In [2]:
import os
import pandas as pd
import numpy as np
from rdkit import Chem


DATAPATH = "../data"

In [3]:
df = pd.read_csv(os.path.join(DATAPATH, "OSAMasterList.csv"))
df.head()

,OSA_ID,Salt,Batch,Target,Target % activity,SMILES_parent,Img,Name,InChiKey,VendorID/ LabNotebook ID/Other Code,...,CompTox,Lipidmaps,DrugCentral,Carotenoid,Metabolights,Brenda,rhea,chemicalbook,swisslipids,gsrs
0,OSA_000001,NaN,NaN,MurD,79.6,CN1CCN(CC1)c1ccc(cc1)C#N,#NAME?,4-(4-methylpiperazin-1-yl)benzonitrile,ZSDPKKGOSKXEHN-UHFFFAOYSA-N,Z2856434840,...,DTXSID30354088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OSA_000002,NaN,NaN,MurD,128,CN1CCN(CC1)C(=O)NC1=CC=C(F)C=C1,#NAME?,N-(4-fluorophenyl)-4-methylpiperazine-1-carbox...,MDBPFVSVLGYVCQ-UHFFFAOYSA-N,Z2856434944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OSA_000003,NaN,NaN,MurD,NaN,CC(CO)(CO)NC(=O)Nc1ccccc1,#NAME?,"3-(1,3-dihydroxy-2-methylpropan-2-yl)-1-phenyl...",NLGYHTMGWVQQIL-UHFFFAOYSA-N,Z57472297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OSA_000004,NaN,NaN,MurD,72.4,CC(C(Nc1cc(C#N)ccc1)=O)C,#NAME?,N-(3-cyanophenyl)-2-methylpropanamide,JWBISRKEEZGPFB-UHFFFAOYSA-N,Z26548083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OSA_000005,NaN,NaN,MurE,NaN,O=S1(CCN(CC1)Cc2ccc(C)cc2)=O,#NAME?,"4-(4-methylbenzyl)thiomorpholine 1,1-dioxide",PBEMXBVPRZGFNM-UHFFFAOYSA-N,Z2856434929,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df[["OSA_ID","Target % activity", "SMILES_parent" ]]
df.rename({"OSA_ID":"ID", "Target % activity":"activity", "SMILES_parent":"SMILES"}, axis=1, inplace=True)

In [5]:
df.dropna(subset=["activity"], inplace=True)

In [7]:
act = df["activity"].tolist()

df["activity"] = df["activity"].apply(lambda x: float(x.split(" &")[0]))


In [8]:
bin_act = []
for i in df["activity"].tolist():
    if i > 20:
        bin = 0
    else:
        bin = 1
    bin_act += [bin]
df["bin_act"]=bin_act

In [9]:
len(df[df["bin_act"]==1])
len(df[df["bin_act"]==0])

51

In [10]:
import sys
sys.path.append("..")
from src.lazy_morgan import MorganBinaryClassifier

In [11]:
mdl = MorganBinaryClassifier()
mdl.fit(df["SMILES"], df["bin_act"])

[flaml.automl: 05-28 19:10:43] {2365} INFO - task = classification
[flaml.automl: 05-28 19:10:43] {2367} INFO - Data split method: stratified
[flaml.automl: 05-28 19:10:43] {2371} INFO - Evaluation method: holdout
[flaml.automl: 05-28 19:10:43] {2448} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 05-28 19:10:43] {2561} INFO - List of ML learners in AutoML Run: ['rf']
[flaml.automl: 05-28 19:10:43] {2853} INFO - iteration 0, current learner rf
[flaml.automl: 05-28 19:10:44] {2984} INFO - Estimated sufficient time budget=3703s. Estimated necessary time budget=4s.
[flaml.automl: 05-28 19:10:44] {3036} INFO -  at 0.4s,	estimator rf's best error=0.0625,	best estimator rf's best error=0.0625
[flaml.automl: 05-28 19:10:44] {2853} INFO - iteration 1, current learner rf
[flaml.automl: 05-28 19:10:44] {3036} INFO -  at 0.8s,	estimator rf's best error=0.0625,	best estimator rf's best error=0.0625
[flaml.automl: 05-28 19:10:44] {2853} INFO - iteration 2, current learner rf
[flaml.automl

In [12]:
mdl._auroc

0.9583333333333334